# 013-LCSM

# 課題のコンセプト
- 与えられた複数のDNA配列から、最長共通部分配列を見つける

# 課題の概要
- input: 複数の1kb未満DNA配列, FASTA形式で与えられる
- output: 与えられたすべての配列が持つ部分配列のうち、最長の部分配列

In [25]:
# FASTAをリストに変換する（ヘッダーは必要ない）
def fasta_to_list(fasta:str)->list:
    sequences = []
    seq=""
    with open(fasta) as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if seq:
                    sequences.append(seq)
                seq = ""
            else:
                seq += line
    if seq:
        sequences.append(seq)
    return sequences

In [26]:
fasta_file = "/home/kinari/lab-work/Rosalind/rosalind_lcsm.txt"
sequences = fasta_to_list(fasta_file)
print(sequences)

['...']


# 解法1 
- 与えられた配列の中で最も短いものの中から最長共通部分配列を見つける
- 最長（最短配列の全長）からさかのぼって探索する
    - 少々計算時間が長い(31s)
    - lcs = len(longest_common_substring)
    - ss = len(shortest_seq)
    - n = number of sequence
    ```math
    \sum_{k=lcs}^{ss}(ss-k+1)*n
    ```
    というようになり、計算量が非常に膨大である

In [27]:
def find_longest_common_substring(sequences:list)-> str:
    shortest_seq= min(sequences, key=len)
    longest_common_substring = ""
    for length in reversed(range(1,len(shortest_seq)+1)):
        for i in range(0,len(shortest_seq)-length+1): # range()はendpointを含まない
            match_count = 0
            partial_seq = shortest_seq[i:i+length]
            for seq in sequences:
                if partial_seq in seq:
                    match_count += 1
            if match_count == len(sequences):
                longest_common_substring = partial_seq
        if longest_common_substring:
            return longest_common_substring  # 最初に見つかった最長共通部分文字列を返す
    return ""

In [28]:
longest_common_substring = find_longest_common_substring(sequences)
print(longest_common_substring)

AATATACGTTCCTGAAGAGCTCAATTCCTGCAGTCAGCATCGGCGGTTACATCTATATGGTTTGATAGCGTCAACACTGCCACTCGAGGCGTCGGCACCCGCACGACTTGCGGGCAACCAATCTACCCCCATGGTGTTGAAAACACATATGACGGACAGAGATGAAGATGGGAATGCTCCCGACAACACTGCGACACTTCCCCCATCTATAGTGTGCATGGATGAGCCCCAAACCATTGATTGAATGTGCGTCCATGAGAAGTGGTTGGTCTG


# all()を活用してもっとシンプルにする

In [29]:
def find_longest_common_substring(sequences:list)-> str:
    shortest_seq= min(sequences, key=len)
    longest_common_substring = ""
    for length in reversed(range(1,len(shortest_seq)+1)):
        for i in range(0,len(shortest_seq)-length+1):
            partial_seq = shortest_seq[i:i+length]
            if all(partial_seq in seq for seq in sequences):
                longest_common_substring = partial_seq
        if longest_common_substring:
            return longest_common_substring
    return ""

In [34]:
longest_common_substring = find_longest_common_substring(sequences)
print(longest_common_substring)

AATATACGTTCCTGAAGAGCTCAATTCCTGCAGTCAGCATCGGCGGTTACATCTATATGGTTTGATAGCGTCAACACTGCCACTCGAGGCGTCGGCACCCGCACGACTTGCGGGCAACCAATCTACCCCCATGGTGTTGAAAACACATATGACGGACAGAGATGAAGATGGGAATGCTCCCGACAACACTGCGACACTTCCCCCATCTATAGTGTGCATGGATGAGCCCCAAACCATTGATTGAATGTGCGTCCATGAGAAGTGGTTGGTCTG


# 計算時間の短縮
- substringの長さを最も長いものから計算するのではなく、二分探索する？
   - 必ずしも有効とは言えない
- 高次元のDPテーブルを作成する
   - 入力配列の数が増えすぎると、計算量が爆発する（一度すべての配列のすべての位置の塩基に対して一致を計算する必要があるから）